In [1]:
import json
import re
import requests

import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
plt.style.use('ggplot')
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing  import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from IPython.core.display import HTML, display
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vald_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\vald_\AppData\Local\Temp\ipykernel_24708\1076333182.py:22: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [2]:
SEED = 42

In [3]:
X = pd.read_csv("train.csv")
X.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [4]:
del X["id"]
X.head()

,url,title,target
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [5]:
y = X["target"]
y.head()

0    False
1    False
2    False
3    False
4    False
Name: target, dtype: bool

In [6]:
del X["target"]
X.head()

,url,title
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц..."
1,www.kp.by,Эта песня стала известна многим телезрителям б...
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...
3,colorbox.spb.ru,Не Беси Меня Картинки
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...


### Submit

In [8]:
# test_df["target"] = [("порно" in text) for text in X_test]

# test_df[["id", "target"]].to_csv("simple_baseline.csv", index=False)

# !cat simple_baseline.csv | head

### Не все так однозначно

**не порно**:
- Болезни опорно-двигательной системы и импотенция: взаимосвязь
- Транссексуальные рыбы - National Geographic Россия: красота мира в каждом кадре
- Групповая обзорная экскурсия по Афинам - цена €50
- Больного раком Задорнова затравили в соцсетях.
- Гомосексуалисты на «Первом канале»? Эрнст и Галкин – скрытая гей-пара российского шоу-бизнеса | Заметки о стиле, моде и жизни

**порно**:
- Отборная домашка
- Сюзанна - карьера горничной / Susanna cameriera perversa (с русским переводом) 1995 г., DVDRip

# ML baseline

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.75, random_state=SEED, shuffle=False)

In [93]:
stemmer = nltk.stem.SnowballStemmer(language='russian')

In [8]:
import pymorphy2
lemmatizer = pymorphy2.MorphAnalyzer()

In [9]:
stop_words = set(nltk.corpus.stopwords.words())
len(stop_words)

9380

In [10]:
def preprocessor(text):
    whitespaced_text = re.sub("[^а-яёa-z]", ' ', text.lower())
    clear_text = re.sub(' +', ' ',  whitespaced_text)
    words = []
    for word in clear_text.split():
        inf_word = lemmatizer.parse(word.replace("ё", "е"))[0].normal_form
        if inf_word not in stop_words:
            words.append(inf_word)
    return ' '.join(words)

In [11]:
X["title_clear"] = X["title"].apply(preprocessor)

In [12]:
X["title_words"] = X["title_clear"].apply(str.split)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.75, random_state=SEED, shuffle=False)

In [14]:
def into_pairs(words):
    return [words[i] + '_' + words[i+1] for i in range(len(words)-1)]

In [15]:
from collections import Counter
def get_pairs(words):
    return [(words[i], words[i+1]) for i in range(len(words)-1)]
cooccurence = Counter(pair for words in X_train["title_words"] for pair in get_pairs(words))

In [16]:
MIN_COUNT = 10
step = 1
def find_collocations(words):
    if len(words) < 2:
        return words
    new_words = []
    i = 0
    last_in_pair = False
    while i < len(words) - 1:
        if cooccurence.get((words[i], words[i+1]), 0) >= MIN_COUNT:
            new_words.append(words[i]+"_"+words[i+1])
            i += step
            last_in_pair = True
        else:
            if not last_in_pair:
                new_words.append(words[i])
            i += 1
            last_in_pair = False
    if not last_in_pair:
        new_words.append(words[-1])
    return new_words

In [17]:
X["title_coll"] = X["title_words"].apply(find_collocations)

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.75, random_state=SEED, shuffle=False)

In [57]:
from sklearn.neural_network import MLPClassifier

In [63]:
title_model = Pipeline([
    (
        'vectorizer',
        TfidfVectorizer(
            lowercase=False,
            analyzer='word',
            ngram_range=(1,2),
        )
    ),
    # ('clf', KNeighborsClassifier(n_neighbors=3, metric='cosine'))
    # ('clf', MLPClassifier(alpha=0.00006))
    ('clf', SGDClassifier(random_state=SEED, loss='modified_huber', class_weight='balanced', alpha=0.000007))
])

In [64]:
title_model.fit(X_train["title_clear"], y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(lowercase=False, ngram_range=(1, 2))),
                ('clf',
                 SGDClassifier(alpha=7e-06, class_weight='balanced',
                               loss='modified_huber', random_state=42))])

In [65]:
ypred = title_model.predict(X_val["title_clear"])

In [66]:
f1_score(y_val, ypred)

0.9606017191977078

In [80]:
array = np.array(X_val["title_clear"])

In [144]:
stemmer.stem("мастурбация")

'мастурбац'

In [143]:
lemmatizer.parse("мастурбация")

[Parse(word='мастурбация', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='мастурбация', score=1.0, methods_stack=((DictionaryAnalyzer(), 'мастурбация', 41, 0),))]

In [149]:
mytest = np.array(["мастурбация мастурбация"])

In [150]:
title_model.predict_proba(mytest)

array([[0.65612689, 0.34387311]])

In [115]:
title_model.predict_proba(array[148:149])

array([[0.52176449, 0.47823551]])

In [114]:
X_val["title_clear"][101481 + 148]

'проститутка первомайский большой выбор доска бесплатный объявление проститут'

In [39]:
np.where(y_val != ypred)

(array([   49,   135,   146,   148,   193,   303,   321,   339,   346,
          378,   561,   582,   889,   890,   995,  1056,  1108,  1160,
         1174,  1405,  1408,  1421,  1429,  1676,  1696,  1766,  1780,
         1796,  1867,  2295,  2340,  2380,  2471,  2579,  2757,  2812,
         2948,  3015,  3035,  3154,  3178,  3275,  3380,  3573,  3581,
         3683,  3690,  3766,  3948,  4008,  4017,  4057,  4060,  4076,
         4088,  4097,  4212,  4245,  4317,  4377,  4460,  4570,  4611,
         4701,  4722,  4861,  4883,  5017,  5048,  5098,  5150,  5441,
         5442,  5456,  5463,  5548,  5560,  5576,  5645,  5831,  5963,
         6217,  6247,  6341,  6619,  6685,  6766,  6794,  6814,  6820,
         7007,  7282,  7462,  7575,  7669,  7688,  7698,  7916,  7943,
         7998,  8132,  8192,  8194,  8451,  8534,  8775,  8832,  8935,
         8943,  8989,  9144,  9217,  9386,  9496,  9523,  9580,  9772,
         9814,  9887,  9975,  9998, 10000, 10165, 10348, 10392, 10610,
      

In [28]:
from sklearn.metrics import classification_report

In [38]:
print(classification_report(y_val, ypred))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99     29603
        True       0.97      0.95      0.96      4225

    accuracy                           0.99     33828
   macro avg       0.98      0.97      0.98     33828
weighted avg       0.99      0.99      0.99     33828



In [510]:
f1_score(y_train, title_model.predict(X_train["title_clear"]))

0.9871743486973948

In [822]:
def parse_url(url):
    return ' '.join([lemmatizer.parse(word)[0].normal_form for word in re.split(r'[-_\.]+', url.lower())])

In [841]:
url_model = Pipeline([
    (
        'vectorizer',
        TfidfVectorizer(
            lowercase=True, ngram_range=(1, 1), analyzer='word',
            preprocessor=preprocessor, min_df=3, max_df=0.5
        )
    ),
    ('clf', SGDClassifier(random_state=SEED, loss='modified_huber', class_weight='balanced', alpha=0.00007))
])

In [842]:
url_model.fit(X_train["url"], y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=0.5, min_df=3,
                                 preprocessor=<function preprocessor at 0x00000292D3517D00>)),
                ('clf',
                 SGDClassifier(alpha=7e-05, class_weight='balanced',
                               loss='modified_huber', random_state=42))])

In [843]:
y_pred = url_model.predict(X_val["url"])

In [844]:
f1_score(y_val, y_pred)

0.814887579131194

In [516]:
f1_score(y_train, url_model.predict(X_train["url"]))

0.9511919160391713

In [442]:
X_train

,url_score,title_score
0,0.901196,1.00000
1,1.000000,1.00000
2,0.979404,1.00000
3,0.938890,0.97144
4,1.000000,1.00000
...,...,...
101476,0.938102,1.00000
101477,1.000000,1.00000
101478,1.000000,1.00000
101479,1.000000,1.00000


In [ ]:
# k-Neighbors Classifier
# Support Vector Machines
# Decision Tree Classifier
# Random Forest Classifier
# Gradient Boosting Classifier
# Logistic Regression
# Naive Bayes Classifier
# Neural Network Classifier
# Linear Discriminant Analysis Classifier
# Quadratic Discriminant Analysis Classifier

In [344]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [517]:
X_train

,url,title,title_clear,title_words,title_coll
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",экс министр экономика молдова глава мидэя цель...,"[экс, министр, экономика, молдова, глава, мидэ...","[экс, министр, экономика, молдова, глава, мидэ..."
1,www.kp.by,Эта песня стала известна многим телезрителям б...,песня стать известный многий телезритель благо...,"[песня, стать, известный, многий, телезритель,...","[песня, стать_известный, многий, телезритель, ..."
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,банша сезон серия бремя красота смотреть онлайн,"[банша, сезон, серия, бремя, красота, смотреть...","[банша, сезон_серия, бремя, красота_смотреть, ..."
3,colorbox.spb.ru,Не Беси Меня Картинки,бесить картинка,"[бесить, картинка]","[бесить, картинка]"
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,новомосковск сыграть следж хоккеист алексински...,"[новомосковск, сыграть, следж, хоккеист, алекс...","[новомосковск, сыграть, следж, хоккеист, алекс..."
...,...,...,...,...,...
101476,medbrowse.com.ua,"НЕФРОФІТ 100 цена и наличие в аптеках Черкасс,...",нефрофа цена наличие аптека черкассы купить не...,"[нефрофа, цена, наличие, аптека, черкассы, куп...","[нефрофа, цена_наличие, наличие_аптека, черкас..."
101477,manga-online.biz,Читать мангу Данганронпа Гайден: Убийца убийц ...,читать манга данганронп гайден убийца убийца г...,"[читать, манга, данганронп, гайден, убийца, уб...","[читать_манга, данганронп, гайден, убийца_убий..."
101478,podarok52.ru,Фоторамки багет пластик — Каталог — Страница 2...,фоторамка багет пластик каталог страница подар...,"[фоторамка, багет, пластик, каталог, страница,...","[фоторамка, багет, пластик, каталог_страница, ..."
101479,www.warandpeace.ru,Польский гонор и русская честь - ВОЙНА и МИР,польский гонор русский честь война мир,"[польский, гонор, русский, честь, война, мир]","[польский, гонор, русский, честь, война_мир]"


In [118]:
vec_size = 300

In [120]:
word2vec = gensim.models.Word2Vec(
    vector_size=vec_size, sg=0, window=5, min_count=3
)

In [122]:
word2vec.build_vocab(X_train["title_words"], keep_raw_vocab=True)

In [123]:
word2vec.train(X_train["title_words"], total_examples=word2vec.corpus_count, epochs=30)

(19900377, 23824950)

In [133]:
word2vec.wv.most_similar("мастурбация страница".split())

[('оргазм', 0.6685698628425598),
 ('рогоносец', 0.6149997711181641),
 ('молоденький', 0.6067529916763306),
 ('порнозвезда', 0.5997006297111511),
 ('сквирт', 0.5875810384750366),
 ('вебкамера', 0.5826252102851868),
 ('рвать', 0.5797131657600403),
 ('porus', 0.5784225463867188),
 ('проникновение', 0.5782161951065063),
 ('страстный', 0.577751636505127)]

In [134]:
def list_to_vec(words_list):
    known_words = 0
    vec = np.array([0] * vec_size)
    if not len(words_list):
        return vec
    for word in words_list:
        try:
            v = word2vec.wv.get_vector(word)
            vec = vec + v
            known_words += 1
        except:
            continue
    if known_words == 0:
        return vec
    return vec / known_words

In [153]:
def get_similar_words(word):
    try:
        return [sim for sim, poss in word2vec.wv.most_similar(word, topn=3) if poss > 0.6]
    except:
        return []

In [ ]:
def get_new_text(text):
    new_words = []
    text = text.split()
    [for word in text.split() for v in list_to_vec(word.split(), 4) if ]

In [151]:
v = list_to_vec("мастурбация".split())

In [152]:
word2vec.wv.most_similar(v)

[('мастурбация', 0.9999999403953552),
 ('сквирт', 0.8152335286140442),
 ('оргазм', 0.7998698353767395),
 ('чпокаяться', 0.7878572940826416),
 ('дрочко', 0.7782406806945801),
 ('толстушка', 0.7712351083755493),
 ('дилдый', 0.7705534100532532),
 ('мастурбировать', 0.7652242183685303),
 ('дрочки', 0.7635612487792969),
 ('волосатый', 0.7629032731056213)]

In [622]:
def into_pairs(words):
    return [words[i] + '_' + words[i+1] for i in range(len(words)-1)]

In [623]:
X["title_w2v"] = X["title_coll"].apply(list_to_vec)

In [624]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.75, random_state=SEED, shuffle=False)

In [626]:
from sklearn.neighbors import KNeighborsClassifier

In [660]:
def linear_weights(distance):
    k = distance.shape[1]
    return ((k - np.arange(0, k))/k) * np.ones_like(distance)

In [674]:
w2v_model = KNeighborsClassifier(n_neighbors=5, weights=linear_weights)

In [675]:
w2v_model.fit(np.vstack(X_train["title_w2v"]), y_train)

KNeighborsClassifier(weights=<function linear_weights at 0x000002932A7385E0>)

In [676]:
pred = w2v_model.predict(np.vstack(X_val["title_w2v"]))

In [677]:
f1_score(y_val, pred)

0.9300758213984835

In [694]:
X["w2v_score"] = w2v_model.predict_proba(np.vstack(X["title_w2v"]))[:, 0]

In [830]:
X["url_score"] = url_model.predict_proba(X["url"])[:, 0]

In [697]:
X["title_score"] = title_model.predict_proba(X["title_clear"])[:, 0]

In [831]:
X_train, X_val, y_train, y_val = train_test_split(X[["url_score", "title_score", "w2v_score"]], y, train_size=0.75, 
                                                  random_state=SEED, shuffle=False)

In [836]:
main_clf = SGDClassifier(random_state=SEED, loss='modified_huber', class_weight='balanced', alpha=0.00007)
# main_clf = LogisticRegression()
# main_clf = GradientBoostingClassifier()
# main_clf = MLPClassifier(alpha=0.00006)


In [837]:
main_clf.fit(X_train, y_train)

SGDClassifier(alpha=7e-05, class_weight='balanced', loss='modified_huber',
              random_state=42)

In [838]:
y_pred = main_clf.predict(X_val)

In [839]:
f1_score(y_val, y_pred)

0.9782019559326028

,url_score,title_score,w2v_score
0,0.901196,1.00000,1.000000
1,1.000000,1.00000,0.866667
2,0.979404,1.00000,1.000000
3,0.938890,0.97144,1.000000
4,1.000000,1.00000,1.000000
...,...,...,...
101476,0.938102,1.00000,1.000000
101477,1.000000,1.00000,1.000000
101478,1.000000,1.00000,1.000000
101479,1.000000,1.00000,1.000000


In [ ]:
# main_clf = SGDClassifier(random_state=SEED, loss='log', class_weight='balanced', alpha=0.00007)
# main_clf = LogisticRegression(tol=0.001)
# main_clf =`` GradientBoostingClassifier()
main_clf = MLPClassifier(alpha=0.00006)


In [ ]:
main_clf.fit(X_train, y_train)

MLPClassifier(alpha=6e-05)

In [ ]:
y_pred = main_clf.predict(X_val)

In [ ]:
f1_score(y_val, y_pred)

0.9803266960772624

In [ ]:
# main_clf = SGDClassifier(random_state=SEED, loss='log', class_weight='balanced', alpha=0.00007)
# main_clf = LogisticRegression(tol=0.001)
# main_clf =`` GradientBoostingClassifier()
main_clf = MLPClassifier(alpha=0.00006)


In [ ]:
# main_clf = SGDClassifier(random_state=SEED, loss='log', class_weight='balanced', alpha=0.00007)
# main_clf = LogisticRegression(tol=0.001)
# main_clf =`` GradientBoostingClassifier()
main_clf = MLPClassifier(alpha=0.00006)


In [ ]:
main_clf.fit(X_train, y_train)

MLPClassifier(alpha=6e-05)

In [ ]:
y_pred = main_clf.predict(X_val)

In [ ]:
f1_score(y_val, y_pred)

0.9803266960772624

In [ ]:
main_clf.fit(X_train, y_train)

MLPClassifier(alpha=6e-05)

In [ ]:
y_pred = main_clf.predict(X_val)

In [ ]:
f1_score(y_val, y_pred)

0.9803266960772624

CountVectorizer

In [322]:
TOKEN_PATTERN = "[а-яёa-z]+"

In [329]:
count_model = Pipeline([
    (
        'vectorizer',
        CountVectorizer(
            lowercase=True, ngram_range=(1, 1), token_pattern=TOKEN_PATTERN,
            stop_words=stop_words, min_df=3
        )
    ),
    ('clf', SGDClassifier(random_state=SEED, loss='log', class_weight='balanced'))
])

In [327]:
count_model.fit(Xt_train, y_train)

c:\Users\vald_\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ac', 'acaba', 'ace', 'aceast', 'aib', 'ais', 'aix', 'aldiz', 'altm', 'ambd', 'amos', 'aqu', 'aras', 'arr', 'art', 'asl', 'ave', 'avr', 'azut', 'b', 'baiknya', 'baizik', 'bamos', 'ban', 'baz', 'bb', 'bel', 'ber', 'berkali', 'birka', 'bizl', 'br', 'bukatzeko', 'bunlar', 'cadasc', 'dac', 'dana', 'dant', 'despr', 'dou', 'dup', 'dur', 'dvain', 'edota', 'ega', 'eiv', 'ejo', 'elel', 'elela', 'elele', 'eleli', 'elelo', 'elen', 'elena', 'elene', 'eleni', 'eleno', 'eleti', 'eli', 'elijo', 'elim', 'elimo', 'elita', 'elite', 'eliva', 'emer', 'emerkoli', 'emo', 'emu', 'emur', 'emurkoli', 'enkoli', 'eprav', 'eravno', 'erega', 'eremu', 'erih', 'erim', 'erima', 'erimi', 'esar', 'esarkoli', 'estdeset', 'estdeseta', 'estdesete', 'estdesetega', 'estdesetem', 'estdesetemu', 'es

Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.8, min_df=3,
                                 stop_words=['إذ', 'إذا', 'إذما', 'إذن', 'أف',
                                             'أقل', 'أكثر', 'ألا', 'إلا',
                                             'التي', 'الذي', 'الذين', 'اللاتي',
                                             'اللائي', 'اللتان', 'اللتيا',
                                             'اللتين', 'اللذان', 'اللذين',
                                             'اللواتي', 'إلى', 'إليك', 'إليكم',
                                             'إليكما', 'إليكن', 'أم', 'أما',
                                             'أما', 'إما', 'أن', ...],
                                 token_pattern='[а-яёa-z]+')),
                ('clf',
                 SGDClassifier(class_weight='balanced', loss='log',
                               random_state=42))])

In [328]:
f1_score(y_test, count_model.predict(Xt_test))

0.4155354449472097

In [367]:
vectorizer = CountVectorizer()
model = MultinomialNB()

In [203]:
X.head()

,url,title
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц..."
1,www.kp.by,Эта песня стала известна многим телезрителям б...
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...
3,colorbox.spb.ru,Не Беси Меня Картинки
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...


In [368]:
vectorizer.fit(X["title"])
Xt_train_vectorized = vectorizer.transform(Xt_train)
Xt_test_vectorized = vectorizer.transform(Xt_test)

In [369]:
%%time

model.fit(
    Xt_train_vectorized,
    y_train
)

y_pred = model.predict(
    Xt_test_vectorized
)

CPU times: total: 31.2 ms
Wall time: 34 ms


In [370]:
f1_score(y_test, y_pred)

0.6354833904364666

In [256]:
parameters = {
    'vectorizer__max_df': (0.5, 0.75),
    'vectorizer__min_df': (3, 5, 7),
    'clf__alpha': (0.0001, 0.001, 0.01),
}

In [ ]:
grid_search = GridSearchCV(count_model, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=1)

TFIDF Vectorizer

Word2vec

In [386]:
X["title"].apply(preprocessor)

0         экс министр экономики молдовы главе мидэи цель...
1         эта песня стала известна многим телезрителям б...
2          банши сезон серия бремя красоты смотреть онлайн 
3                                     не беси меня картинки
4         в новомосковске сыграют следж хоккеисты алекси...
                                ...                        
135304    пора тюльпанов турецкий сериал на русском язык...
135305    остросюжетный сериал шеф игра на повышение я с...
135306     plymouth special deluxe hot rod automatic sma...
135307    купить it s skin сыворотка питательная power f...
135308    технический спирт в канистрах и флаконах купит...
Name: title, Length: 135309, dtype: object

### Submit

In [17]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["target"] = model.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: stdout: Broken pipe
